# Thinking Model Logging Example
This notebook provides a quick summary of what you would get from logging a multimodal model call from a model like Claude.

In [0]:
%pip install openai mlflow
%restart_python

## Inference without thinking budget
Here is inference without a thinking budget, where we simply get a response.

In [0]:
from openai import OpenAI
import os
import base64
from PIL import Image
import mlflow
import io

DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Path to image file in Databricks volume
image_path = "./image.png"

# Read and encode the image
with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode("utf-8")

mlflow.openai.autolog()

client = OpenAI(
  api_key=DATABRICKS_TOKEN,
  base_url="https://adb-984752964297111.11.azuredatabricks.net/serving-endpoints"
)

chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": """
            Question: What is the missing color of the part denoted with a question mark?
          
            Options: (A) Purple (B) Green (C) Blue (D) Yellow
            
            Answer: Let's describe the image first and think step by step.
            """},
        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}}
      ]
    }
  ],
  model="databricks-claude-3-7-sonnet",
  max_tokens=512
)

parsed_text = chat_completion.choices[0].message.content
print(parsed_text)

## Inference with thinking enabled
The response varies a bit when you enable thinking mode. Note the difference in the logging - we get the entire message stream back because the output format is slightly different, but once again it is only logged in a single step because MLflow can't introspect within the model.

In [0]:
mlflow.openai.autolog()

client = OpenAI(
  api_key=DATABRICKS_TOKEN,
  base_url="https://adb-984752964297111.11.azuredatabricks.net/serving-endpoints"
)

chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": """
            Question: What is the missing color of the part denoted with a question mark?
          
            Options: (A) Purple (B) Green (C) Blue (D) Yellow
            
            Answer: Let's describe the image first and think step by step.
            """},
        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}}
      ]
    }
  ],
  model="databricks-claude-3-7-sonnet",
  max_tokens=1500,
  extra_body={
      "thinking": {
          "type": "enabled",
          "budget_tokens": 1024,
      }
  },
)

parsed_text = chat_completion.choices[0].message.content
print(parsed_text)